In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D, SimpleRNN
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
from keras.utils import to_categorical

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
import nltk
import re
from nltk.tokenize import sent_tokenize

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
rcv1_train = pd.read_csv("/content/drive/MyDrive/ReutersTextClassification/Datasets/RCV1/rcv1_train_data_formatted.csv")
rcv1_test=pd.read_csv("/content/drive/MyDrive/ReutersTextClassification/Datasets/RCV1/rcv1_test_data_formatted.csv")

codes=[]
with open('/content/drive/MyDrive/ReutersTextClassification/Datasets/RCV1/rcv1.topics.txt','r') as f:
  lines=f.readlines()
  for line in lines:
    codes.append(line.rstrip())
print(len(codes))
print(codes)


rcv1_train_labels = rcv1_train[codes]
rcv1_test_labels = rcv1_test[codes]

Y = rcv1_train_labels.values
Y_test = rcv1_test_labels.values

103
['C11', 'C12', 'C13', 'C14', 'C15', 'C151', 'C1511', 'C152', 'C16', 'C17', 'C171', 'C172', 'C173', 'C174', 'C18', 'C181', 'C182', 'C183', 'C21', 'C22', 'C23', 'C24', 'C31', 'C311', 'C312', 'C313', 'C32', 'C33', 'C331', 'C34', 'C41', 'C411', 'C42', 'CCAT', 'E11', 'E12', 'E121', 'E13', 'E131', 'E132', 'E14', 'E141', 'E142', 'E143', 'E21', 'E211', 'E212', 'E31', 'E311', 'E312', 'E313', 'E41', 'E411', 'E51', 'E511', 'E512', 'E513', 'E61', 'E71', 'ECAT', 'G15', 'G151', 'G152', 'G153', 'G154', 'G155', 'G156', 'G157', 'G158', 'G159', 'GCAT', 'GCRIM', 'GDEF', 'GDIP', 'GDIS', 'GENT', 'GENV', 'GFAS', 'GHEA', 'GJOB', 'GMIL', 'GOBIT', 'GODD', 'GPOL', 'GPRO', 'GREL', 'GSCI', 'GSPO', 'GTOUR', 'GVIO', 'GVOTE', 'GWEA', 'GWELF', 'M11', 'M12', 'M13', 'M131', 'M132', 'M14', 'M141', 'M142', 'M143', 'MCAT']


In [ ]:
print('average train sentence length: ', rcv1_train.newsitem.str.split().str.len().mean())
print('stdev train sentence length: ', rcv1_train.newsitem.str.split().str.len().std())

average train sentence length:  222.89727418031018
stdev train sentence length:  201.0941646168312


In [ ]:
print('average test sentence length: ', rcv1_test.newsitem.str.split().str.len().mean())
print('stdev test sentence length: ', rcv1_test.newsitem.str.split().str.len().std())

average test sentence length:  230.09108351754406
stdev test sentence length:  208.92326731703892


In [ ]:
MAX_WORDS_PER_SENT = 100
MAX_SENT = 15
MAX_VOC_SIZE = 20000
#GLOVE_DIM = 100
TEST_SPLIT = 0.2

In [ ]:
# Do some basic cleaning of the review text
def remove_quotations(text):
    """
    Remove quotations and slashes from the dataset.
    """
    text = re.sub(r"\\", "", str(text))
    text = re.sub(r"\'", "", str(text))
    text = re.sub(r"\"", "", str(text))
    return text


In [ ]:
rcv1_train['newsitem'] = rcv1_train['newsitem'].apply(remove_quotations)
rcv1_test['newsitem'] = rcv1_test['newsitem'].apply(remove_quotations)

rcv1_train['newsitem'] = rcv1_train['newsitem'].apply(lambda x: x.strip().lower())
rcv1_test['newsitem'] = rcv1_test['newsitem'].apply(lambda x: x.strip().lower())

In [ ]:
import gc
# Get the data and the sentiment
news = rcv1_train['newsitem'].values
target = rcv1_train_labels.values

news_test = rcv1_test['newsitem'].values
target_test = rcv1_test_labels.values

del rcv1_train
del rcv1_test
gc.collect()

124

In [ ]:
# Build a Keras Tokenizer that can encode every token
word_tokenizer = Tokenizer(num_words=MAX_VOC_SIZE)
word_tokenizer.fit_on_texts(news)

In [ ]:
# Construct the input matrix. This should be a nd-array of
# shape (n_samples, MAX_SENT, MAX_WORDS_PER_SENT).
# We zero-pad this matrix (this does not influence
# any predictions due to the attention mechanism.
X = np.zeros((len(news), MAX_SENT, MAX_WORDS_PER_SENT), dtype='int32')

In [ ]:
#for test set
X_test = np.zeros((len(news_test), MAX_SENT, MAX_WORDS_PER_SENT), dtype='int32')

In [ ]:
nltk.download('punkt')
for i, single_news in enumerate(news):
    sentences = sent_tokenize(single_news)
    tokenized_sentences = word_tokenizer.texts_to_sequences(
        sentences
    )
    tokenized_sentences = pad_sequences(
        tokenized_sentences, maxlen=MAX_WORDS_PER_SENT
    )

    pad_size = MAX_SENT - tokenized_sentences.shape[0]

    if pad_size < 0:
        tokenized_sentences = tokenized_sentences[0:MAX_SENT]
    else:
        tokenized_sentences = np.pad(
            tokenized_sentences, ((0, pad_size), (0, 0)),
            mode='constant', constant_values=0
        )

    # Store this observation as the i-th observation in
    # the data matrix
    X[i] = tokenized_sentences[None, ...]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#for test set
for i, single_news in enumerate(news_test):
    sentences = sent_tokenize(single_news)
    tokenized_sentences_test = word_tokenizer.texts_to_sequences(
        sentences
    )
    tokenized_sentences_test = pad_sequences(
        tokenized_sentences_test, maxlen=MAX_WORDS_PER_SENT
    )

    pad_size = MAX_SENT - tokenized_sentences_test.shape[0]

    if pad_size < 0:
        tokenized_sentences_test = tokenized_sentences_test[0:MAX_SENT]
    else:
        tokenized_sentences_test = np.pad(
            tokenized_sentences_test, ((0, pad_size), (0, 0)),
            mode='constant', constant_values=0
        )

    # Store this observation as the i-th observation in
    # the data matrix
    X_test[i] = tokenized_sentences_test[None, ...]

In [ ]:
# Transform the labels into a format Keras can handle
y = np.asarray(target)

In [ ]:
# Transform the labels into a format Keras can handle
y_test = np.asarray(target_test)

In [ ]:
# We make a train/test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=TEST_SPLIT)

In [ ]:
word_index = word_tokenizer.word_index
max_features = len(word_index)+1
def load_glove(word_index):
    EMBEDDING_FILE = '/content/drive/MyDrive/ReutersTextClassification/WordEmbedding/Glove_6B/glove.6B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

def load_fasttext(word_index):    
    EMBEDDING_FILE = '/content/drive/MyDrive/ReutersTextClassification/WordEmbedding/Fasttext/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100 and o.split(" ")[0] in word_index )

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100 and o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix






In [ ]:
embedding_matrix = load_glove(word_index)
#embedding_matrix_2 = load_fasttext(word_index)
#embedding_matrix_3 = load_para(word_index)
#embedding_matrix = np.mean((embedding_matrix_1, embedding_matrix_3), axis=0)  
#del embedding_matrix_1, embedding_matrix_3
gc.collect()
np.shape(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


(77961, 300)

In [ ]:
from tensorflow import keras
class AttentionLayer(keras.layers.Layer):
    def __init__(self, context_vector_length=100, **kwargs):
        """
        An implementation of a attention layer. This layer
        accepts a 3d Tensor (batch_size, time_steps, input_dim) and
        applies a single layer attention mechanism in the time
        direction (the second axis).
        :param context_vector_lenght: (int) The size of the hidden context vector.
            If set to 1 this layer reduces to a standard attention layer.
        :param kwargs: Any argument that the baseclass Layer accepts.
        """
        self.context_vector_length = context_vector_length
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        dim = input_shape[2]

        # Add a weights layer for the
        self.W = self.add_weight(
            name='W', shape=(dim, self.context_vector_length),
            initializer=keras.initializers.get('uniform'),
            trainable=True
        )

        self.u = self.add_weight(
            name='context_vector', shape=(self.context_vector_length, 1),
            initializer=keras.initializers.get('uniform'),
            trainable=True
        )

        super(AttentionLayer, self).build(input_shape)

    def _get_attention_weights(self, X):
        """
        Computes the attention weights for each timestep in X
        :param X: 3d-tensor (batch_size, time_steps, input_dim)
        :return: 2d-tensor (batch_size, time_steps) of attention weights
        """
        # Compute a time-wise stimulus, i.e. a stimulus for each
        # time step. For this first compute a hidden layer of
        # dimension self.context_vector_length and take the
        # similarity of this layer with self.u as the stimulus
        u_tw = K.tanh(K.dot(X, self.W))
        tw_stimulus = K.dot(u_tw, self.u)

        # Remove the last axis an apply softmax to the stimulus to
        # get a probability.
        tw_stimulus = K.reshape(tw_stimulus, (-1, tw_stimulus.shape[1]))
        att_weights = K.softmax(tw_stimulus)

        return att_weights

    def call(self, X):
        att_weights = self._get_attention_weights(X)

        # Reshape the attention weights to match the dimensions of X
        att_weights = K.reshape(att_weights, (-1, att_weights.shape[1], 1))
        att_weights = K.repeat_elements(att_weights, X.shape[-1], -1)

        # Multiply each input by its attention weights
        weighted_input = keras.layers.Multiply()([X, att_weights])

        # Sum in the direction of the time-axis.
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

    def get_config(self):
        config = {
            'context_vector_length': self.context_vector_length
        }
        base_config = super(AttentionLayer, self).get_config()
        return {**base_config, **config}

In [ ]:
class HAN(Model):
    def __init__(
            self, max_words, max_sentences, output_size,
            embedding_matrix, word_encoding_dim=300,
            sentence_encoding_dim=300, inputs=None,
            outputs=None, name='han-for-docla'
    ):
        """
        A Keras implementation of Hierarchical Attention networks
        for document classification.
        :param max_words: The maximum number of words per sentence
        :param max_sentences: The maximum number of sentences
        :param output_size: The dimension of the last layer (i.e.
            the number of classes you wish to predict)
        :param embedding_matrix: The embedding matrix to use for
            representing words
        :param word_encoding_dim: The dimension of the GRU
            layer in the word encoder.
        :param sentence_encoding_dim: The dimension of the GRU
            layer in the sentence encoder.
        """
        self.max_words = max_words
        self.max_sentences = max_sentences
        self.output_size = output_size
        self.embedding_matrix = embedding_matrix
        self.word_encoding_dim = word_encoding_dim
        self.sentence_encoding_dim = sentence_encoding_dim

        in_tensor, out_tensor = self._build_network()

        super(HAN, self).__init__(
            inputs=in_tensor, outputs=out_tensor, name=name
        )

    def build_word_encoder(self, max_words, embedding_matrix, encoding_dim=200):
        """
        Build the model that embeds and encodes in context the
        words used in a sentence. The return model takes a tensor of shape
        (batch_size, max_length) that represents a collection of sentences
        and returns an encoded representation of these sentences.
        :param max_words: (int) The maximum sentence length this model accepts
        :param embedding_matrix: (2d array-like) A matrix with the i-th row
            representing the embedding of the word represented by index i.
        :param encoding_dim: (int, should be even) The dimension of the
            bidirectional encoding layer. Half of the nodes are used in the
            forward direction and half in the backward direction.
        :return: Instance of keras.Model
        """
        assert encoding_dim % 2 == 0, "Embedding dimension should be even"

        vocabulary_size = embedding_matrix.shape[0]
        embedding_dim = embedding_matrix.shape[1]

        embedding_layer = Embedding(
            vocabulary_size, embedding_dim,
            weights=[embedding_matrix], input_length=max_words,
            trainable=True
        )

        sentence_input = Input(shape=(max_words,), dtype='int32')
        embedded_sentences = embedding_layer(sentence_input)
        encoded_sentences = Bidirectional(
            GRU(int(encoding_dim / 2), return_sequences=True)
        )(embedded_sentences)

        return Model(
            inputs=[sentence_input], outputs=[encoded_sentences], name='word_encoder'
        )

    def build_sentence_encoder(self, max_sentences, summary_dim, encoding_dim=200):
        """
        Build the encoder that encodes the vector representation of
        sentences in their context.
        :param max_sentences: The maximum number of sentences that can be
            passed. Use zero-padding to supply shorter sentences.
        :param summary_dim: (int) The dimension of the vectors that summarizes
            sentences. Should be equal to the encoding_dim of the word
            encoder.
        :param encoding_dim: (int, even) The dimension of the vector that
            summarizes sentences in context. Half is used in forward direction,
            half in backward direction.
        :return: Instance of keras.Model
        """
        assert encoding_dim % 2 == 0, "Embedding dimension should be even"

        text_input = Input(shape=(max_sentences, summary_dim))
        encoded_sentences = Bidirectional(
            GRU(int(encoding_dim / 2), return_sequences=True)
        )(text_input)
        return Model(
            inputs=[text_input], outputs=[encoded_sentences], name='sentence_encoder'
        )

    def _build_network(self):
        """
        Build the graph that represents this network
        :return: in_tensor, out_tensor, Tensors representing the input and output
            of this network.
        """
        in_tensor = Input(shape=(self.max_sentences, self.max_words))

        word_encoder = self.build_word_encoder(
            self.max_words, self.embedding_matrix, self.word_encoding_dim
        )

        word_rep = TimeDistributed(
            word_encoder, name='word_encoder'
        )(in_tensor)

        # Sentence Rep is a 3d-tensor (batch_size, max_sentences, word_encoding_dim)
        sentence_rep = TimeDistributed(
            AttentionLayer(), name='word_attention'
        )(word_rep)

        doc_rep = self.build_sentence_encoder(
            self.max_sentences, self.word_encoding_dim, self.sentence_encoding_dim
        )(sentence_rep)

        # We get the final representation by applying our attention mechanism
        # to the encoded sentences
        doc_summary = AttentionLayer(name='sentence_attention')(doc_rep)

        out_tensor = Dense(
            self.output_size, activation='sigmoid', name='class_prediction'
        )(doc_summary)

        return in_tensor, out_tensor

    def get_config(self):
        config = {
            'max_words': self.max_words,
            'max_sentences': self.max_sentences,
            'output_size': self.output_size,
            'embedding_matrix': self.embedding_matrix,
            'word_encoding_dim': self.word_encoding_dim,
            'sentence_encoding_dim': self.sentence_encoding_dim,
            'base_config': super(HAN, self).get_config()
        }

        return config

    @classmethod
    def from_config(cls, config, custom_objects=None):
        """
        Keras' API isn't really extendible at this point
        therefore we need to use a bit hacky solution to
        be able to correctly reconstruct the HAN model
        from a config. This therefore does not reconstruct
        a instance of HAN model, but actually a standard
        Keras model that behaves exactly the same.
        """
        base_config = config.pop('base_config')

        return Model.from_config(
            base_config, custom_objects=custom_objects
        )

    def predict_sentence_attention(self, X):
        """
        For a given set of texts predict the attention
        weights for each sentence.
        :param X: 3d-tensor, similar to the input for predict
        :return: 2d array (num_obs, max_sentences) containing
            the attention weights for each sentence
        """
        att_layer = self.get_layer('sentence_attention')
        prev_tensor = att_layer.input

        # Create a temporary dummy layer to hold the
        # attention weights tensor
        dummy_layer = Lambda(
            lambda x: att_layer._get_attention_weights(x)
        )(prev_tensor)

        return Model(self.input, dummy_layer).predict(X)

In [ ]:
han_model = HAN(
    MAX_WORDS_PER_SENT, MAX_SENT, len(codes), embedding_matrix,
    word_encoding_dim=300, sentence_encoding_dim=300
)

In [ ]:
han_model.summary()

Model: "han-for-docla"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15, 100)]         0         
_________________________________________________________________
word_encoder (TimeDistribute (None, 15, 100, 300)      23795100  
_________________________________________________________________
word_attention (TimeDistribu (None, 15, 300)           30100     
_________________________________________________________________
sentence_encoder (Functional (None, 15, 300)           406800    
_________________________________________________________________
sentence_attention (Attentio (None, 300)               30100     
_________________________________________________________________
class_prediction (Dense)     (None, 103)               31003     
Total params: 24,293,103
Trainable params: 24,293,103
Non-trainable params: 0
_________________________________________

In [ ]:

han_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

filepath='/content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=1, min_lr=0.0001, verbose=2)
earlystopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto')

callbacks = [checkpoint, earlystopping, reduce_lr]

print("model fitting - Hierachical attention network")
history=han_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100, batch_size=64,callbacks=callbacks)




model fitting - Hierachical attention network
Epoch 1/100
579/579 [==============================] - 989s 2s/step - loss: 0.6317 - accuracy: 2.1290e-05 - val_loss: 0.4663 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.46627, saving model to /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f/assets


Epoch 2/100
579/579 [==============================] - 1149s 2s/step - loss: 0.4228 - accuracy: 4.9382e-05 - val_loss: 0.3126 - val_accuracy: 0.0050

Epoch 00002: val_loss improved from 0.46627 to 0.31262, saving model to /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f/assets


...

Epoch 22/100
579/579 [==============================] - 1319s 2s/step - loss: 0.1080 - accuracy: 0.0042 - val_loss: 0.1081 - val_accuracy: 0.0052

Epoch 00022: val_loss improved from 0.10841 to 0.10807, saving model to /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f/assets


Epoch 23/100
424/579 [====================>.........] - ETA: 4:57 - loss: 0.1071 - accuracy: 0.0046

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Models/text_cnn_500_trainabe_true_fasttext.hd5f',custom_objects={"HAN":HAN,"AttentionLayer":AttentionLayer})

In [ ]:
predictions=model.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
#delta, _ = bestThresshold(Y_test,predictions)

In [ ]:
pred_bools = [pl>0.50 for pl in predictions] #boolean output after thresholding

#Print and save classification report
print('Test F1 Accuracy(Micro): ', f1_score(y_test, pred_bools,average='micro'))
print('Test F1 Accuracy(Macro): ', f1_score(y_test, pred_bools,average='macro'))
print('Test Flat Accuracy: ', accuracy_score(y_test, pred_bools),'\n')

In [ ]:
m = tf.keras.metrics.Precision(top_k=1)
m.update_state(Y_test, pred_bools)
m.result().numpy()

In [ ]:
m = tf.keras.metrics.Precision(top_k=3)
m.update_state(Y_test, pred_bools)
m.result().numpy()

In [ ]:
m = tf.keras.metrics.Precision(top_k=5)
m.update_state(Y_test, pred_bools)
m.result().numpy()